In [1]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 4.2 MB/s 
     |████████████████████████████████| 880 kB 56.2 MB/s 
     |████████████████████████████████| 6.6 MB 46.5 MB/s 
     |████████████████████████████████| 77 kB 5.8 MB/s 
     |████████████████████████████████| 596 kB 53.0 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=2255d8a08cedc682f507a20c77f031ab1770b156050eff35cb75aba256422d66
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# !pip install sentencepiece

     |████████████████████████████████| 1.2 MB 4.2 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')
%cd /content/gdrive/Shareddrives/DL

Mounted at /content/gdrive/
/content/gdrive/Shareddrives/DL


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
import numpy as np
import torch
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel, RobertaForSequenceClassification, BertTokenizer
from transformers import PLBartTokenizer, PLBartForConditionalGeneration
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm

import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
from transformers import T5Tokenizer, T5Model,T5ForConditionalGeneration

# plb_model =  PLBartForConditionalGeneration.from_pretrained("PLBART Model",max_position_embeddings=1500,ignore_mismatched_sizes=True).to(device)
t5_model = T5ForConditionalGeneration.from_pretrained("CodeT5FinalModel",dropout_rate=0.5,d_model=1850,ignore_mismatched_sizes=True).to(device)

t5_model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32100, 1850)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32100, 1850)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1850, out_features=512, bias=False)
              (k): Linear(in_features=1850, out_features=512, bias=False)
              (v): Linear(in_features=1850, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=1850, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.5, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=1850, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=1850, bias=False)
              (dropo

In [8]:
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-small')

def RobertaTokenizerForSeq2SeqTest(x):
  return tokenizer.batch_encode_plus(x,max_length=1000,truncation=True,padding='max_length',return_tensors="pt", add_special_tokens = True)

Downloading:   0%|          | 0.00/687k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/287k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/12.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

In [9]:
tc2code = pd.read_json('./data.jsonl',lines=True)

In [10]:
tc2code.head()

,testCode,sourceCode
0,/*\n * This file was automatically generated b...,package macaw.util;\n\nimport macaw.system.Mac...
1,/*\n * This file was automatically generated b...,package macaw.util;\n\nimport macaw.system.Use...
2,/*\n * This file was automatically generated b...,package macaw.util;\n\nimport java.util.regex....
3,/*\n * This file was automatically generated b...,package macaw.util;\n\nimport java.awt.event.C...
4,/*\n * This file was automatically generated b...,package macaw.util;\n\nimport javax.swing.*;\n...


In [11]:
from sklearn.model_selection import train_test_split
_, X_test, _, y_test = train_test_split(tc2code['testCode'], tc2code['sourceCode'], test_size=0.2, shuffle=False)
_, X_test, _, y_test = train_test_split(X_test, y_test, test_size=0.1, shuffle=False)

In [12]:
print(len(X_test.to_list()))

459


In [14]:
print(type(RobertaTokenizerForSeq2SeqTest(X_test.to_list()[1:2])))

<class 'transformers.tokenization_utils_base.BatchEncoding'>


In [15]:
print(y_test.to_list()[0])

/*
 * Created on 14/mar/2010
 * Copyright (C) 2010 by Andrea Vacondio.
 *
 * This program is free software; you can redistribute it and/or modify it under the terms of the 
 * GNU General Public License as published by the Free Software Foundation; 
 * either version 2 of the License.
 * This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; 
 * without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. 
 * See the GNU General Public License for more details.
 * You should have received a copy of the GNU General Public License along with this program; 
 * if not, write to the Free Software Foundation, Inc., 
 *  59 Temple Place, Suite 330, Boston, MA 02111-1307 USA
 */
package org.pdfsam.guiclient.business.listeners;

import java.awt.event.InputEvent;
import java.awt.event.MouseWheelEvent;
import java.awt.event.MouseWheelListener;

import org.pdfsam.guiclient.gui.components.JPreviewImage;

/**
 * Listen for the mouse whe

In [16]:
from torch.utils import data
class CodeTCDatasetTest(data.Dataset):
    def __init__(self, testTensor,expectedSourceCode,testCodeInput):
      self.testTensor=testTensor
      self.expectedSourceCode=expectedSourceCode
      self.testCodeInput=testCodeInput


    def __getitem__(self, index):
        return self.testTensor[index],self.expectedSourceCode[index],self.testCodeInput[index]


    def __len__(self):
        return len(self.testCodeInput)


def collate_fn(data):
    testTensorList,expectedSourceCode,testCodeInput = zip(*data)

    testTensors = torch.stack(testTensorList, 0)
      
    return testTensors, expectedSourceCode, testCodeInput

def get_test_loader(testTensor,expectedSourceCode,testCodeInput,batch_size):
    code2tctest = CodeTCDatasetTest(testTensor,expectedSourceCode,testCodeInput)
    data_loader = torch.utils.data.DataLoader(dataset=code2tctest, 
                                              batch_size=batch_size,
                                              collate_fn=collate_fn)
    return data_loader

In [17]:
X_test_tensors=RobertaTokenizerForSeq2SeqTest(X_test.to_list())
input_ids_test = X_test_tensors['input_ids']

In [18]:
print(len(input_ids_test))

459


In [19]:
batch_size=64
test_loader=get_test_loader(input_ids_test,y_test.to_list(),X_test.to_list(),batch_size)

In [20]:
print(type(X_test_tensors))

<class 'transformers.tokenization_utils_base.BatchEncoding'>


In [37]:
def getPredictionsGreedy(inputs,model,device):
      greedy_output = model.generate(inputs.to(device), max_length=1800)
      sequences=[]
      for i in range(greedy_output.shape[0]):
        sequences.append(tokenizer.decode(greedy_output[i], skip_special_tokens=True))
      return sequences

def getPredictionsTopK(inputs,model,device):
      sample_outputs = model.generate(
        inputs.to(device),
        do_sample=True, 
        max_length=1800, 
        top_k=900, 
        top_p=0.95, 
        num_return_sequences=3)
    # outputs = model.generate(input_ids)
      sequences=[]
      for i in range(sample_outputs.shape[0]):
        sequences.append(tokenizer.decode(sample_outputs[i], skip_special_tokens=True))
      return sequences

In [38]:
def getPredictedTexts(dataloader_test,genAlgo,model,device,targIters=1,need_targ_iters=False):
    predictions=[]
    allExpectedprogs=[]
    allIpProgs=[]
    idx=0
    with torch.no_grad():
      for ip,estr,istr in dataloader_test:
          if idx>=targIters and need_targ_iters:
            break
          print(f"iter{idx} of {len(dataloader_test)}")
          prediction=genAlgo(ip,model,device)
          predictions=predictions+prediction
          expected=[x for x in estr]
          inputs=[x for x in istr]
          allExpectedprogs=allExpectedprogs+expected
          allIpProgs=allIpProgs+inputs
          idx+=1
    return allExpectedprogs,predictions,allIpProgs

In [26]:
allExpectedprogsGreedy,predictionsGreedy,ipTestCode=getPredictedTexts(test_loader,getPredictionsGreedy,t5_model,device)

iter0 of 15
iter1 of 15
iter2 of 15
iter3 of 15
iter4 of 15
iter5 of 15
iter6 of 15
iter7 of 15
iter8 of 15
iter9 of 15
iter10 of 15
iter11 of 15
iter12 of 15
iter13 of 15
iter14 of 15


In [27]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [28]:
print(allExpectedprogsGreedy[12])

/*
 * Created on 06-Sep-2009
 * Copyright (C) 2009 by Andrea Vacondio.
 *
 * This program is free software; you can redistribute it and/or modify it under the terms of the 
 * GNU General Public License as published by the Free Software Foundation; 
 * either version 2 of the License.
 * This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; 
 * without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. 
 * See the GNU General Public License for more details.
 * You should have received a copy of the GNU General Public License along with this program; 
 * if not, write to the Free Software Foundation, Inc., 
 *  59 Temple Place, Suite 330, Boston, MA 02111-1307 USA
 */
package org.pdfsam.guiclient.business.thumbnails.callables;

import java.awt.image.BufferedImage;
import java.util.concurrent.Callable;

import org.apache.log4j.Logger;
import org.icepdf.core.pobjects.Document;
import org.icepdf.core.pobjects.PDimension;
i

In [29]:
print(ipTestCode[12])

/*
 * This file was automatically generated by EvoSuite
 */

package org.pdfsam.guiclient.business.thumbnails.callables;

import org.junit.Test;
import org.junit.runner.RunWith;
import org.evosuite.junit.EvoSuiteRunner;
import static org.junit.Assert.*;
import org.icepdf.core.pobjects.Document;
import org.junit.BeforeClass;
import org.pdfsam.guiclient.business.thumbnails.callables.IcePdfThumbnailCallable;
import org.pdfsam.guiclient.commons.panels.JVisualPdfPageSelectionPanel;
import org.pdfsam.guiclient.dto.VisualPageListItem;

@RunWith(EvoSuiteRunner.class)
public class IcePdfThumbnailCallableEvoSuiteTest {

  @BeforeClass 
  public static void initEvoSuiteFramework(){ 
    org.evosuite.Properties.REPLACE_CALLS = true; 
  } 


  @Test
  public void test0()  throws Throwable  {
      Document document0 = new Document();
      IcePdfThumbnailCallable icePdfThumbnailCallable0 = new IcePdfThumbnailCallable(document0, (VisualPageListItem) null, (JVisualPdfPageSelectionPanel) null, (-1L));

In [30]:
print(predictionsGreedy[12])

/*
 * Copyright (C) 2001-2009 VTT Technical Research Centre of Finland.
 *
 *
 * This library is free software; you can redistribute it and/or
 * modify it under the terms of the GNU Lesser General Public
 * License as published by the Free Software Foundation;
 * version 2.1 of the License.
 *
 * This library is distributed in the hope that it will be useful,
 *
 * but WITHOUT ANY WARRANTY; without even the implied warranty of
 * MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU
 * Lesser General Public License for more details.
 *
 *
 * * You should should should should should should should should should should should should should should should should should should should should should should should should should should
 *
 *
 * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 

In [31]:
print(predictionsGreedy[52])

/*
 * Copyright (C) 2001-2009 VTT Technical Research Centre of Finland.
 *
 *
 * This library is free software; you can redistribute it and/or
 * modify it under the terms of the GNU Lesser General Public
 * License as published by the Free Software Foundation;
 * version 2.1 of the License.
 *
 * This library is distributed in the hope that it will be useful,
 *
 * but WITHOUT ANY WARRANTY; without even the implied warranty of
 * MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU
 * Lesser General Public License for more details.
 *
 *
 * * You should should should should should should should should should should should should should should should should should should should should should should should should should should
 *
 *
 * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 

Predictions are super bad, so how does t5 fare with top 5

In [32]:
predictiondf_t5 = pd.DataFrame(list(zip(allExpectedprogsGreedy, predictionsGreedy,ipTestCode)), columns =['Expected', 'predictions','ipUnitTest']) 

In [33]:
predictiondf_t5.to_json('ResultValidation/predictions_codet5.jsonl',orient='records', lines=True)

In [39]:
allExpectedprogsGreedy,predictionsGreedy,ipTestCode=getPredictedTexts(test_loader,getPredictionsTopK,t5_model,device,1,True)
predictiondf_t5 = pd.DataFrame(list(zip(allExpectedprogsGreedy, predictionsGreedy,ipTestCode)), columns =['Expected', 'predictions','ipUnitTest']) 
predictiondf_t5.to_json('ResultValidation/predictions_codet5_set2.jsonl',orient='records', lines=True)

iter0 of 15


In [40]:
predictionsGreedy[0]

'/*\n * This file is part of the the the..\n * * * * * Copyright (C) 2001---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Still bad :(